<img src="http://static1.squarespace.com/static/55d78486e4b038548bc9f33e/t/55ec5931e4b0518639da93c6/1448895140273/?format=1500w"/>

## Training on Joined Data

In this notebook we will demonstrate how to join pandas DataFrames.  We will use this technique to join the accident and person tables from the FARS data set.  We will then train on the joined data.

### Loading the Data

Nothing much to see here.  We are loading the tables that we will work with as we have done many times before, using the pandas read_csv method.

In [21]:
import pandas as pd
import numpy as np

acc_df = pd.read_csv('../../data/train/accident_train.csv')
per_df = pd.read_csv('../../data/train/person_train.csv')
veh_df = pd.read_csv('../../data/train/vehicle_train.csv')

/Users/Chengyu/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Dealing with Shared Columns

Both the accident and person tables have several columns in common.  We want to join on just the 'ID' column.  When pandas sees columns with the same name, it makes a copy of one of them and then adds it to the joined DataFrame.  This adds redundant information to our data, and so we will avoid it by dropping the duplicate columns from one of the tables.

In [22]:
# Get a list of each set of columns, and instantiate a new list that will contain the shared columns.
acc_cols = acc_df.columns.tolist()
per_cols = per_df.columns.tolist()
shared_cols = []

# Go through each column in the accident table.  If the column is also in the person table, add
# it to the list.
for col in acc_cols:
    if col in per_cols:
        shared_cols.append(col)

# ID is in both tables, but we will want to use it, so don't include it in the list of columns
# to be removed.
shared_cols.remove('ID')

### Joining DataFrames in Pandas

Pandas has a merge method that will join two DataFrames on a specified column.  We specify which columna and what type of join to use.

In [23]:
# Create a new joined DataFrame.
# Note that we are dropping the shared columns from the person DataFrame before joining it to
# the accident DataFrame.

joined_df = pd.merge(acc_df, per_df.drop(shared_cols, axis=1), on='ID', how='inner')

In [24]:
joined_df[0:20]

,ID,STATE,VE_FORMS,PEDS,PERSONS,COUNTY,CITY,YEAR,DAY,MONTH,...,DEATH_HR,DEATH_MN,DEATH_TM,LAG_HRS,LAG_MINS,CERT_NO,WORK_INJ,HISPANIC,RACE,LOCATION
0,0,1,1,0,1,21,0,2003,1,1,...,9,5,905,1,35,************,9,7,1,0
1,1,1,1,0,1,71,0,2003,1,1,...,16,44,1644,0,54,************,0,7,1,0
2,2,1,1,0,1,51,0,2003,5,1,...,13,35,1335,1,16,************,0,7,2,0
3,3,1,1,0,1,111,0,2003,4,1,...,8,25,825,0,35,************,0,1,1,0
4,4,1,2,0,3,13,0,2003,1,1,...,0,0,0,999,99,************,8,0,0,0
5,4,1,2,0,3,13,0,2003,1,1,...,19,30,1930,0,0,************,0,7,2,0
6,4,1,2,0,3,13,0,2003,1,1,...,0,0,0,999,99,************,8,0,0,0
7,5,1,1,0,1,101,0,2003,11,1,...,17,0,1700,0,40,************,0,7,1,0
8,6,1,2,0,3,43,0,2003,4,1,...,0,0,0,999,99,************,8,0,0,0
9,6,1,2,0,3,43,0,2003,4,1,...,18,0,1800,0,0,************,0,7,7,0


In [25]:
# Here we do a little bit more clean up, replacing missing data with 0, and removing the
# 'CERT_NO' column, which is useless.  
# We decided to make a bad_cols list so that more columns could be removed if you choose.

bad_cols = ['CERT_NO']
joined_df.drop(bad_cols, axis=1, inplace=True)
joined_df.fillna(0, inplace=True)

### Training on Joined Data

Everything has been smooth sailing so far, but we are about to run into a wrinkle.  How should we train on the joined data?  Many of the accidents now have several rows describing them.  How should we deal with these duplicates?  How do we build a train_test_split on this data?

For training, we will want to consider all the data about each accident in its entirety.  Thus, the pandas train_test_split method is insufficient for splitting this data for validation, as it may split a single accident into the training set and the test set.  We address this with the following voodoo.

In [ ]:
grouped = joined_df.groupby('ID')
pd.DataFrame(grouped)[0:20]

In [33]:
# First, get the index of each accident, and choose a subset of them for a train_test_split.
indices = acc_df.ID.values
num_indices = len(indices)
test_size = int(num_indices*0.75)

perm_indices = np.random.permutation(indices)
train_indices = perm_indices[:test_size]
test_indices = perm_indices[test_size:]

# We will now group each accident by ID.
grouped = joined_df.groupby('ID')

# The grouped DataFrame has a dictionary attribute called indices.
#
# This dictionary maps an ID to the indices that correspond to the ID 
# in the joined table.  Thus, we can build new dictionaries that map
# the ids in the training set to the corresponding rows in the joined table,
# and similarly for the test set.

tr_ids = {key : grouped.indices[key] for key in train_indices}
te_ids = {key : grouped.indices[key] for key in test_indices}

# Finally, we turn the dictionaries we created above into lists of row numbers.
# We can then use these row numbers to build our train/test split.
train_ids = []
for key in tr_ids:
    for ind in tr_ids[key].tolist():
        train_ids.append(ind)

test_ids = []
for key in te_ids:
    for ind in te_ids[key].tolist():
        test_ids.append(ind)        

In [34]:
# Build the Xtrain and Xtest DataFrames.

Xtrain = joined_df.ix[train_ids]
Xtest = joined_df.ix[test_ids]

In [35]:
# Build the Ytrain and Ytest DataFrames.
Ytrain = Xtrain[['ID', 'DRUNK_DR']]
Ytest = Xtest[['ID', 'DRUNK_DR']]

# Drop some columns that aren't going to help us.
Xtrain.drop(['DRUNK_DR', 'RAIL', 'TWAY_ID', 'YEAR'], axis=1, inplace=True)
Xtest.drop(['DRUNK_DR', 'RAIL', 'TWAY_ID', 'YEAR'], axis=1, inplace=True)

In [36]:
Xtr = Xtrain.drop('ID', axis=1).values
Ytr = Ytrain['DRUNK_DR'].values
join_cols = Xtrain.columns.tolist()

In [37]:
Xte = Xtest.drop('ID', axis=1).values
Yte = Ytest['DRUNK_DR'].values

In [38]:
# Hopefully all this work has paid off.  Build a RandomForestClassifier, train
# a model, and make a prediction about the test set.

from sklearn.ensemble import RandomForestClassifier

rf1 = RandomForestClassifier()
rf1.fit(Xtr, Ytr)

yhat1 = rf1.predict_proba(Xte)

### Consolidating Predictions on Joined Data

We have one more wrinkle to iron out.  We have made a prediction for each row in the test set.  However, many accidents span several rows.  To evaluate our prediction, we need to make only one prediction per accident.

There are several ways that we might do this.  Here, we group the predictions by ID, and then attach the mean of the predictions to each ID.

In [39]:
predict_df = pd.DataFrame({'ID':Ytest['ID'].values, 'prob':yhat1[:,1]})
grouped_predict = predict_df.groupby('ID')
prediction = grouped_predict.mean()

In [40]:
# This cell builds a RandomForestClassifier on the unmerged data, as we did
# in our Getting Started With Fars notebook.

acc_df.fillna(0, inplace=True)
Xtrain = acc_df.ix[tr_ids].sort_index()
Xtest = acc_df.ix[te_ids].sort_index()

Ytrain = Xtrain[['ID', 'DRUNK_DR']]
Ytest = Xtest[['ID', 'DRUNK_DR']]

Xtrain.drop(['DRUNK_DR', 'RAIL', 'TWAY_ID', 'YEAR'], axis=1, inplace=True)
Xtest.drop(['DRUNK_DR', 'RAIL', 'TWAY_ID', 'YEAR'], axis=1, inplace=True)

Xtr = Xtrain.drop('ID',axis=1).values
Ytr = Ytrain['DRUNK_DR'].values
vanilla_cols = Xtrain.columns.tolist()

rf2 = RandomForestClassifier()
rf2.fit(Xtr, Ytr)

Xte = Xtest.drop('ID',axis=1).values

yhat2 = rf2.predict_proba(Xte)

In [41]:
# Here we get the target that we were tryin to predict in order to make a comparison.
Y = acc_df.ix[test_indices]['DRUNK_DR']
Y = Y.sort_index()

In [42]:
# Calculate the AUC score for our preditions.
from sklearn.metrics import roc_auc_score as auc

auc1 = auc(Y.astype(int), prediction.values)
auc2 = auc(Y.astype(int), yhat2[:,1])

auc1, auc2

(0.82140707662144863, 0.7843366006725111)

## What's Next?

We have seen that adding the person data to the accident data yields a better prediction.  From here, we should do several things:

1. Make a prediction on the entire training set and submit it to Kaggle.
2. Incorporate the vehicle data to our models.
3. Further clean the data.
4. Tune the parameters on the RandomForestClassifier to see if you can get improvement.
5. Try different models to see if they perform better than the RandomForestClassifier.

In [43]:
join_cols = join_cols[1:]
vanilla_cols = vanilla_cols[1:]

feat_imps1 = np.argsort(rf1.feature_importances_).tolist()
feat_imps2 = np.argsort(rf2.feature_importances_).tolist()

In [45]:
print('Most Important Features for Vanilla Data')
for feat in feat_imps2[:20]:
    print('\t'+vanilla_cols[feat])

print('Most Important Features for Joined Data')
for feat in feat_imps1[:20]:
    print('\t'+join_cols[feat])

Most Important Features for Vanilla Data
	SCH_BUS
	CF3
	CF2
	SP_JUR
	CF1
	FATALS
	VE_FORMS
	NHS
	WEATHER
	HOSP_HR
	MAN_COLL
	HOSP_MN
	NOT_HOUR
	PEDS
	ROUTE
	PERSONS
	CITY
	NOT_MIN
	ROAD_FNC
	ARR_MIN
Most Important Features for Joined Data
	CF3
	EMER_USE
	CF2
	SCH_BUS
	SPEC_USE
	FIRE_EXP
	SP_JUR
	LOCATION
	TOW_VEH
	WORK_INJ
	EXTRICAT
	PER_TYP
	CF1
	HISPANIC
	RACE
	EJECTION
	SEX
	PER_NO
	HOSPITAL
	DOA


In [182]:
len(rf2.feature_importances_), len(vanilla_cols)

(34, 35)